<a href="https://colab.research.google.com/github/imukoki/NLP-FELLOWSHIP/blob/Week-7/RNN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Model Assignment
The aim of this assignment is to make sure you understand the RNN model famility. We have covered RNN, LSTM and GRU. You task is 
1. to write code for training any of the RNNs ML model that gives the highest accuracy


## This we will consider
1. The code works
2. The hyperparameter used for fine tuning - epoch only is not enough
3. The highest accuracy
4. Bonus points for being creative with preprocessing, tokenization and creation of vectors

# Submissions
1. Notebook with code
2. 1 models

# Deadline
Monday 5th

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir("/content/gdrive/MyDrive/NLP-FELLOWSHIP/Week 7/")
!ls

'1_Sequence_to_Sequence_Learning_with_Neural_Networks (1).ipynb'
 amazon_reviews.csv
 amazon_reviews.gsheet
 anna.txt
'Copy of Character_Level_RNN_Solution.ipynb'
'NLPNN (2).ipynb'
 RNN_Assignment.ipynb


The dataset that was used for this assignment was found on Kaggle, the name of the dataset is [amazon reviews](https://www.kaggle.com/discussions/general/338657)

In [3]:
import pandas as pd

full_dataset = pd.read_csv('amazon_reviews.csv')
full_dataset.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [4]:
# Selecting only the reviewText and overall columns
dataset = full_dataset[['reviewText', 'overall']]
dataset.head()

,reviewText,overall
0,No issues.,4.0
1,"Purchased this for my device, it worked as adv...",5.0
2,it works as expected. I should have sprung for...,4.0
3,This think has worked out great.Had a diff. br...,5.0
4,"Bought it with Retail Packaging, arrived legit...",5.0


In [5]:
for ind,rating in enumerate(dataset['overall']):
  if rating >= 3:
    dataset['overall'][ind] = 1
  else:
    dataset['overall'][ind] = 0
dataset.head()


<ipython-input-5-fa0c26ecc879>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['overall'][ind] = 1
<ipython-input-5-fa0c26ecc879>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['overall'][ind] = 0


,reviewText,overall
0,No issues.,1.0
1,"Purchased this for my device, it worked as adv...",1.0
2,it works as expected. I should have sprung for...,1.0
3,This think has worked out great.Had a diff. br...,1.0
4,"Bought it with Retail Packaging, arrived legit...",1.0


In [6]:
print("No of positive : "+str(len(dataset[dataset['overall'] == 1])))
print("No of negative : "+str(len(dataset[dataset['overall'] == 0])))


No of positive : 4591
No of negative : 324


In [7]:
split = []
for ind in range(len(dataset)):
  if ind % 2 == 0:
    split.append('train')
  else:
    split.append('test')

dataset['set'] = split
dataset.head()

<ipython-input-7-7df2d4e9b070>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['set'] = split


,reviewText,overall,set
0,No issues.,1.0,train
1,"Purchased this for my device, it worked as adv...",1.0,test
2,it works as expected. I should have sprung for...,1.0,train
3,This think has worked out great.Had a diff. br...,1.0,test
4,"Bought it with Retail Packaging, arrived legit...",1.0,train


In [8]:
print("No of positive in train : "+str(len(dataset[(dataset['overall'] == 1) & (dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(dataset[(dataset['overall'] == 0) & (dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(dataset[(dataset['overall'] == 1) & (dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(dataset[(dataset['overall'] == 0) & (dataset['set'] == 'test')])))

No of positive in train : 2304
No of negative in train : 154
No of positive in test : 2287
No of negative in test : 170


In [9]:
dataset['reviewText'].describe()

count           4914
unique          4912
top       No issues.
freq               2
Name: reviewText, dtype: object

In [10]:
dataset = dataset.drop_duplicates(subset=['reviewText'])

In [11]:
dataset['reviewText'].describe()

count           4912
unique          4912
top       No issues.
freq               1
Name: reviewText, dtype: object

In [12]:
print("No of positive in train : "+str(len(dataset[(dataset['overall'] == 1) & (dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(dataset[(dataset['overall'] == 0) & (dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(dataset[(dataset['overall'] == 1) & (dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(dataset[(dataset['overall'] == 0) & (dataset['set'] == 'test')])))

No of positive in train : 2303
No of negative in train : 154
No of positive in test : 2286
No of negative in test : 170


In [13]:
dataset['overall'] = dataset['overall'].astype('int')
dataset['reviewText'] = dataset['reviewText'].astype('str')
dataset.rename(columns={'overall':'sentiment','reviewText':'review'}, inplace = True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4913 entries, 0 to 4914
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     4913 non-null   object
 1   sentiment  4913 non-null   int64 
 2   set        4913 non-null   object
dtypes: int64(1), object(2)
memory usage: 153.5+ KB


In [14]:
train_dataset = dataset[dataset['set'] == 'train'][['review','sentiment']]
test_dataset = dataset[dataset['set'] == 'test'][['review','sentiment']]
test_dataset.head()

,review,sentiment
1,"Purchased this for my device, it worked as adv...",1
3,This think has worked out great.Had a diff. br...,1
5,It's mini storage. It doesn't do anything els...,1
7,It's hard to believe how affordable digital ha...,1
9,"in my galaxy s4, super fast card, and am total...",1


In [15]:
len(test_dataset)

2456

In [16]:
import re
def preprocessing(texts):
  cleaned_text = []
  for text in texts:
    text = text.lower()
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    text = emoji_pattern.sub(r'', text)
    text = url_pattern.sub(r'', text)
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text)
    cleaned_text.append(text)

  return cleaned_text

In [17]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 7.5 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1


In [18]:
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html


In [19]:
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator


SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # Check this
max_document_length = 300 #hyperparameter

TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',preprocessing=preprocessing,batch_first=True,  fix_length=max_document_length)
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )



/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [20]:
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=test_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split() #Text to explain training, testing and validation datasets

In [21]:
torch_train_dataset = DataFrameDataset(
    df=train_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

In [22]:
max_size = 30000 #hyperparameter
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)



In [23]:
vocab_size

6352

In [24]:
print(TEXT.vocab.freqs.most_common(20))

[(' ', 16903), ('i', 4495), ('the', 4270), ('it', 3771), ('and', 3691), ('to', 2939), ('a', 2721), ('my', 2456), ('this', 2246), ('card', 2139), ('for', 1991), ('in', 1649), ('of', 1602), ('is', 1515), ('with', 1505), ('have', 1118), ('on', 1058), ('that', 981), ('was', 845), ('phone', 812)]


In [25]:
BATCH_SIZE = 56 #hyperparameter

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True)#,
    #device = device)

In [26]:
import torch.nn as nn
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNN, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer output layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.float()
        
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
       

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0)
        
        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [30]:
lr = 1e-4
batch_size = 70
dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size
seed = 42
num_classes = 2

num_epochs = 3
hidden_size = 256 # hyperparameter
hidden_size1 = 300
hidden_size2 = 128
hidden_size3 = 64
num_layers = 10

to_train = True

model = RNN(max_document_length, hidden_size1, num_layers, num_classes,dropout_keep_prob)

In [28]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [31]:
best_valid_loss = float('inf')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

loss_func = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  train_epoch_loss = 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      #predictions = model(text).squeeze(1)
      predictions = model(text.unsqueeze(1))
      
      loss = loss_func(predictions, batch.sentiment)

      acc = accuracy(predictions, batch.sentiment)

      # perform backpropagation
      loss.backward()

      optimizer.step()

      train_epoch_loss += loss.item()
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval()

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          #predictions = model(text).squeeze(1)
          predictions = model(text.unsqueeze(1))
          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_rnn.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')


	Train Loss: 0.415 | Train Acc: 93.48%
	 Val. Loss: 0.243 |  Val. Acc: 93.32%
	Train Loss: 0.240 | Train Acc: 93.65%
	 Val. Loss: 0.239 |  Val. Acc: 93.32%
	Train Loss: 0.219 | Train Acc: 93.65%
	 Val. Loss: 0.231 |  Val. Acc: 93.32%
